In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

In [4]:
!pip install tqdm

In [1]:
import pandas as pd
from tqdm import tqdm

## TED

In [2]:
df_train_transcript = pd.read_csv('./CORPUS/MERGE/TRAIN_ALL.csv')
df_train_transcript = df_train_transcript.to_dict(orient='dict')


TRAIN_ESENT = (list(df_train_transcript['EN'].values()))
TRAIN_KSENT = (list(df_train_transcript['KO'].values())) #+ (list(df_hanja_transcript['WORD'].values()))
TRAIN_ZSENT = (list(df_train_transcript['ZH-TW'].values())) #+ (list(df_hanja_transcript['HANJA'].values()))
df_train_transcript = []

In [3]:
df_test_transcript = pd.read_csv('./CORPUS/MERGE/TEST_ALL.csv')
df_test_transcript = df_test_transcript.to_dict(orient='dict')
TEST_ESENT = (list(df_test_transcript['EN'].values()))
TEST_KSENT = (list(df_test_transcript['KO'].values()))
TEST_ZSENT = (list(df_test_transcript['ZH-TW'].values()))
df_test_transcript = []

## HANJA

### HANJ DICTIONARY

In [3]:
df_hanja_transcript = pd.read_csv('./CORPUS/HANJA/KO_HANJA_DICT_ALL_V2.csv')
df_hanja_transcript = df_hanja_transcript.to_dict(orient='dict')

HANJA_KSENT = (list(df_hanja_transcript['WORD'].values()))
HANJA_ZSENT = (list(df_hanja_transcript['HANJA'].values()))

In [4]:
hanja_dict = {}
for i in range(len(HANJA_KSENT)):
  key = HANJA_KSENT[i]
  hanjas = HANJA_ZSENT[i].split('\n')
  hanja_dict[key] = hanjas
print(len(hanja_dict))

23151


In [11]:
print(list(hanja_dict.keys())[0],list(hanja_dict.values())[0])

안녕 ['安寧', '晏寧']


## TED2020

In [51]:
f1 = open('./CORPUS/ted2020/TED2020.ko-zh_tw.ko','r')
ted2020_ko = f1.read().split('\n')
f1.close()

f1 = open('./CORPUS/ted2020/TED2020.ko-zh_tw.zh_tw','r')
ted2020_zh_tw = f1.read().split('\n')
f1.close()

In [50]:
def remove_item(alist,astr):
  res = [it for it in alist if it != astr]
  return res

In [53]:
print(len(ted2020_ko))
print(len(ted2020_zh_tw))

392977
392977


In [60]:
# Remove lines without anything.
ted2020_ko = remove_item(ted2020_ko,'')
ted2020_zh_tw = remove_item(ted2020_zh_tw,'')
print(len(ted2020_ko))
print(len(ted2020_zh_tw))

389275
389275


# Tokenizer

In [6]:
import torch
from transformers import BertTokenizer, BertModel, BertConfig, BertForPreTraining, AutoModel, BertTokenizerFast, RobertaTokenizer, RobertaModel, AutoTokenizer

In [6]:
!pip install dragonmapper
!pip install jieba
!pip install konlpy
!pip install paddlepaddle

UnboundLocalError: local variable 'child' referenced before assignment

#### Zhuyin tokens

In [7]:
from dragonmapper import hanzi
from dragonmapper import transcriptions
import re

In [15]:
zhuyins = ['ㄅ','ㄆ','ㄇ','ㄈ','ㄉ','ㄊ','ㄋ','ㄌ','ㄍ','ㄎ','ㄏ','ㄐ','ㄑ','ㄒ','ㄓ','ㄔ','ㄕ','ㄖ','ㄗ','ㄘ','ㄙ','ㄧ','ㄨ','ㄩ','ㄚ','ㄛ','ㄜ','ㄝ','ㄞ','ㄟ','ㄠ','ㄡ','ㄢ','ㄣ','ㄤ','ㄥ','ㄦ','ˇ','ˋ','ˊ','˙']

In [17]:
def check_bpmf_is_not(text):
    test_list = ['.*[ㄅㄆㄇㄈㄉㄊㄋㄌㄍㄎㄏㄐㄑㄒㄓㄔㄕㄖㄗㄘㄙㄧㄨㄩㄚㄛㄜㄝㄞㄟㄠㄡㄢㄣㄤㄥㄦˇˋˊ˙]+.*']
    for t in test_list:
        valid = re.compile(t)
        try:
            if(valid.match(text) != None):
                return True
        except:
            return False
    return False

In [20]:
print(check_bpmf_is_not('（'))

False


In [21]:
zhuyin_df = pd.read_csv('./CORPUS/TOKEN/zhuyin_tokens_count.csv')
zhuyin_df = zhuyin_df.to_dict(orient='dict')
zhuyin_tokens = list(zhuyin_df['zhuyin'].values())
print(len(zhuyin_tokens))

2751


In [23]:
for tok in zhuyin_tokens:
  if(check_bpmf_is_not(tok)):
    continue
  else:
    zhuyin_tokens.remove(tok)
print(len(zhuyin_tokens))

2089


In [24]:
print(zhuyin_tokens[0])

ㄒㄧㄝˋ


#### Korean tokens

In [26]:
# tokenize with konlpy
ko_token_df = pd.read_csv('./CORPUS/TOKEN/KO_TOKEN_FREQ.csv')
ko_token_df = ko_token_df.to_dict(orient='dict')
ko_tokens = list(ko_token_df['TOKEN'].values())
print(len(ko_tokens))

63964


In [55]:
def check_ko_is_not(text):
    test_list = ['.*[a-z]+.*','.*[A-Z]+.*','.*[0-9]+.*','.*[℃℉!?@$%^&*()-+=.,~`:;]+.*']
    for t in test_list:
        valid = re.compile(t)
        try:
            if(valid.match(text) != None):
                return False
        except:
            return False
    if(len(text)<1):
      return False
    return True

In [39]:
for tok in ko_tokens:
  if(check_ko_is_not(tok)):
    continue
  else:
    ko_tokens.remove(tok)
print(len(ko_tokens))

51768


In [40]:
print(ko_tokens[:20])

['안녕', '하', '세요', '정말', '좋', '지', '않', '습니까', '강연', '들', '이', '너무', '아', '정신', '을', '못', '차리', '겠', '네요', '사실']


#### Tokenizers

In [30]:
ko_tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
#ko_bert_model = BertModel.from_pretrained("kykim/bert-kor-base")

tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

In [42]:
print(len(ko_tokenizer.get_vocab()))

42000


In [59]:
ko_tokenizer_tokens = list(ko_tokenizer.get_vocab().keys())

In [60]:
zh_tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
print('Original Length: ',len(zh_tokenizer.get_vocab()))

zh_tokenizer.add_tokens(zhuyins)
print('Add zhuyins: ',len(zh_tokenizer.get_vocab()))

zh_tokenizer.add_tokens(zhuyin_tokens)
print('Add zhuyin tokens: ',len(zh_tokenizer.get_vocab()))

zh_tokenizer.add_tokens(ko_tokens)
print('Add korean tokens from konlpy: ',len(zh_tokenizer.get_vocab()))

zh_tokenizer.add_tokens(ko_tokenizer_tokens)
print('Add korean tokens from kykim/bert-kor-base tokenizer: ',len(zh_tokenizer.get_vocab()))
#zh_bert_model = AutoModel.from_pretrained('ckiplab/bert-base-chinese')

Original Length:  21128
Add zhuyins:  21146
Add zhuyin tokens:  22839
Add korean tokens from konlpy:  74455
Add korean tokens from kykim/bert-kor-base tokenizer:  105229


In [61]:
zh_tokenizer.save_pretrained('./MODELS/TOKENIZER/new_zh_tokenizer_add_bpmf_korean_tokens')

('/content/drive/MyDrive/ZH-KO-TRANSLATOR/MODELS/TOKENIZER/new_zh_tokenizer_add_bpmf_korean_tokens/tokenizer_config.json',
 '/content/drive/MyDrive/ZH-KO-TRANSLATOR/MODELS/TOKENIZER/new_zh_tokenizer_add_bpmf_korean_tokens/special_tokens_map.json',
 '/content/drive/MyDrive/ZH-KO-TRANSLATOR/MODELS/TOKENIZER/new_zh_tokenizer_add_bpmf_korean_tokens/vocab.txt',
 '/content/drive/MyDrive/ZH-KO-TRANSLATOR/MODELS/TOKENIZER/new_zh_tokenizer_add_bpmf_korean_tokens/added_tokens.json',
 '/content/drive/MyDrive/ZH-KO-TRANSLATOR/MODELS/TOKENIZER/new_zh_tokenizer_add_bpmf_korean_tokens/tokenizer.json')

# Find Hanja in sentences

### Find most suitable hanja

In [8]:
import jieba
jieba.enable_paddle()
jieba.set_dictionary('./CORPUS/TOKEN/jieba_big_dict.txt')

Import error, cannot find paddle.fluid and jieba.lac_small.predict module. Now, back to jieba basic cut......


In [5]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()

In [6]:
def take_tokens(text,lang):
  if lang == 'ko':
    tokens = kkma.pos(text)
    return [tok[0] for tok in tokens]
  else:
    text = text.replace(' ','').replace('\t','')
    tokens = jieba.cut(text,cut_all=False,HMM=True)
    return [tok for tok in tokens]

In [15]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [7]:
from sentence_transformers import SentenceTransformer, util
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer("all-MiniLM-L6-v2").to(device)
model.eval()
print(device)
#K2E_sentence = translator.translate(ko_sentence)
#Z2E_sentence = translator.translate(zh_sentence)

cuda


In [8]:
def find_sentence_pair(model,src_sentence,dst_sentence):
  output_pairs = {}
  src_emb = model.encode(src_sentence)
  dst_emb = model.encode(dst_sentence)
  cos_sim = util.cos_sim(dst_emb,src_emb)
  all_sentence_combinations = []

  for j in range(len(cos_sim)):
    for k in range(len(cos_sim[0])):
      all_sentence_combinations.append([cos_sim[j][k],j,k])

  base_score = cos_sim[0][0]
  #print(base_score)
  all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
  for score, i, j in all_sentence_combinations[0:1]:
    if float(score) > (base_score+0.001):
      #print("{} \t {} \t {:.4f}".format(dst_sentence[0], src_sentence[j], cos_sim[i][j]))
      return [dst_sentence[0],src_sentence[j],float(score)]
  return [dst_sentence[0],src_sentence[0],float(base_score)]

In [9]:
def save_file(ori_sents,hanja_sents,scores,path):
  df = pd.DataFrame({'Original':ori_sents,'Hanja':hanja_sents,'Score':scores})
  df.to_csv('./CORPUS/HANJA/{}.csv'.format(path),index=False)

In [31]:
train_hanja_sents = [] # compare with chinese sentences
train_scores = []

train_hanja_sents_k = [] # compare with original korean sentences
train_scores_k = []


In [32]:
test_hanja_sents = []
test_scores = []

test_hanja_sents_k = []
test_scores_k = []

In [33]:
start = 129500
end = len(TRAIN_ZSENT)

In [34]:
with tqdm(total = len(TRAIN_ZSENT[start:end])) as pbar:
 for i in range(start,end):
  #print(i)
  if(i%100 == 0):
    save_file(TRAIN_KSENT[start:i],train_hanja_sents,train_scores,path='train_compare_with_zh_129500_140000')
    save_file(TRAIN_KSENT[start:i],train_hanja_sents_k,train_scores_k,path='train_compare_with_ko_129500_140000')
  score = 0
  zh_sent = TRAIN_ZSENT[i]
  hanja_sent = TRAIN_KSENT[i]
  hanja_sent_k = TRAIN_KSENT[i]
  sent_i = 0
  ko_tokens = take_tokens(hanja_sent,'ko')
  #print(len(train_hanja_sents),ko_tokens)
  if(i == 129547 or i == 129548):
    
      train_hanja_sents.append(hanja_sent)
      train_scores.append(1)

      train_hanja_sents_k.append(hanja_sent_k)
      train_scores_k.append(1)
      pbar.update(1)
      continue
    

  for tok in ko_tokens:
    new_sents = [hanja_sent]
    new_sents_k = [hanja_sent_k]

    if tok in hanja_dict.keys():
      if(hanja_sent.find(tok)!= -1):
        sent_i = hanja_sent.find(tok, sent_i)
        for han in hanja_dict[tok]:
          try:
            new_ko_sent = hanja_sent[:sent_i] + han + hanja_sent[sent_i+len(tok):]
            new_sents.append(new_ko_sent)

            new_ko_sent_k = hanja_sent_k[:sent_i] + han + hanja_sent_k[sent_i+len(tok):]
            new_sents_k.append(new_ko_sent_k)
          except:
            continue
        #print(tok,hanja_dict[tok])
        #print(new_sents)
        sent_pair_z = find_sentence_pair(model,new_sents,[zh_sent])
        sent_pair_k = find_sentence_pair(model,new_sents_k[1:],[hanja_sent_k])
        #print(sent_pair)

        hanja_sent = sent_pair_z[1]
        score = sent_pair_z[2]
        #print('\nz:',hanja_sent)
        #sent_i += len(tok)

        hanja_sent_k = sent_pair_k[1]
        score_k = sent_pair_k[2]
        #print('k:',hanja_sent_k)
        sent_i += len(tok)
        #print('='*20)

  train_hanja_sents.append(hanja_sent)
  train_scores.append(score)

  train_hanja_sents_k.append(hanja_sent_k)
  train_scores_k.append(score_k)

  #if(i>0):
  #  break
  #print('*'*20)
  pbar.update(1)

save_file(TRAIN_KSENT[start:end],train_hanja_sents,train_scores,'train_compare_with_zh_129500_140000')
save_file(TRAIN_KSENT[start:end],train_hanja_sents_k,train_scores_k,'train_compare_with_ko_129500_140000')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17161/17161 [1:15:27<00:00,  3.79it/s]


In [ ]:
save_file(TRAIN_KSENT,train_hanja_sents,train_scores,'train_compare_with_zh')
save_file(TRAIN_KSENT,train_hanja_sents_k,train_scores_k,'train_compare_with_ko')

In [16]:
with tqdm(total = len(TEST_ZSENT)) as pbar:
 for i in range(0,len(TEST_KSENT)):
  if(i%100 == 0):
    save_file(TEST_KSENT[:i],test_hanja_sents,test_scores,path='test_compare_with_zh')
    save_file(TEST_KSENT[:i],test_hanja_sents_k,test_scores_k,path='test_compare_with_ko')
  score = 0
  zh_sent = TEST_ZSENT[i]
  hanja_sent = TEST_KSENT[i]
  hanja_sent_k = TEST_KSENT[i]
  sent_i = 0
  ko_tokens = take_tokens(hanja_sent,'ko')
  #print(len(train_hanja_sents),ko_tokens)

  for tok in ko_tokens:
    new_sents = [hanja_sent]
    new_sents_k = [hanja_sent_k]

    if tok in hanja_dict.keys():
      if(hanja_sent.find(tok)!= -1):
        sent_i = hanja_sent.find(tok, sent_i)
        for han in hanja_dict[tok]:
          try:
            new_ko_sent = hanja_sent[:sent_i] + han + hanja_sent[sent_i+len(tok):]
            new_sents.append(new_ko_sent)

            new_ko_sent_k = hanja_sent_k[:sent_i] + han + hanja_sent_k[sent_i+len(tok):]
            new_sents_k.append(new_ko_sent_k)
          except:
            continue
        #print(tok,hanja_dict[tok])
        #print(new_sents)
        sent_pair_z = find_sentence_pair(model,new_sents,[zh_sent])
        sent_pair_k = find_sentence_pair(model,new_sents_k[1:],[hanja_sent_k])
        #print(sent_pair)

        hanja_sent = sent_pair_z[1]
        score = sent_pair_z[2]
        #print('\nz:',hanja_sent)
        #sent_i += len(tok)

        hanja_sent_k = sent_pair_k[1]
        score_k = sent_pair_k[2]
        #print('k:',hanja_sent_k)
        sent_i += len(tok)
        #print('='*20)

  test_hanja_sents.append(hanja_sent)
  test_scores.append(score)

  test_hanja_sents_k.append(hanja_sent_k)
  test_scores_k.append(score_k)

  #if(i>0):
  #  break
  #print('*'*20)
  pbar.update(1)

save_file(TEST_KSENT,test_hanja_sents,test_scores,'test_compare_with_zh')
save_file(TEST_KSENT,test_hanja_sents_k,test_scores_k,'test_compare_with_ko')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36089/36089 [2:09:12<00:00,  4.65it/s]


In [ ]:
save_file(TEST_KSENT,test_hanja_sents,test_scores,'test_compare_with_zh')
save_file(TEST_KSENT,test_hanja_sents_k,test_scores_k,'test_compare_with_ko')

In [162]:
print(train_hanja_sents)
print(train_scores)

['感謝합니다.', '동양인 男性 같나요?']
[0.5184196829795837, 0.8020363450050354]


In [163]:
print(train_hanja_sents_k)
print(train_scores_k)

['感謝합니다.', '東洋人 南城 같나요?']
[0.8238508701324463, 0.7364066243171692]


# Phonetic Notation

In [8]:
s = TRAIN_ZSENT[0]
print(hanzi.to_zhuyin(s))

ㄒㄧㄝˋ ㄒㄧㄝˋ ㄉㄚˋ ㄐㄧㄚ。


In [51]:
total_zhuyin_tokens = {}
sent_to_zhuyin = []
error_words = []

In [52]:
with tqdm(total = len(TRAIN_ZSENT)) as pbar:
 for sent in TRAIN_ZSENT:
  #print(sent)
  tokens = zh_tokenizer(sent,max_length=512,truncation=True,)['input_ids']
  #print(tokens)
  sent_zy = ''
  for t in tokens[1:-1]:
    word = zh_tokenizer.decode(t, skip_special_tokens=True)
    try:
      zy = hanzi.to_zhuyin(word)
    except:
      zy = word
      if zy not in error_words:
        error_words.append(zy)
    sent_zy += zy+'\n'
    #print(word,zy)
    try:
      total_zhuyin_tokens[zy] += 1
    except:
      total_zhuyin_tokens[zy] = 1

    #new_tokens = zh_tokenizer(zy)['input_ids']
    #print(new_tokens)
    #new_word = zh_tokenizer.decode(new_tokens, skip_special_tokens=True)
    #print(new_word)
  sent_to_zhuyin.append(sent_zy[:-1])
  pbar.update(1)
  #if(len(sent_to_zhuyin)>2):
  #  break
#print(total_zhuyin_tokens)
#print(sent_to_zhuyin)

100%|██████████| 146661/146661 [06:30<00:00, 375.38it/s]


In [54]:
TEST_sent_to_zhuyin = []

In [55]:
with tqdm(total = len(TEST_ZSENT)) as pbar:
 for sent in TEST_ZSENT:
  #print(sent)
  tokens = zh_tokenizer(sent,max_length=512,truncation=True,)['input_ids']
  #print(tokens)
  sent_zy = ''
  for t in tokens[1:-1]:
    word = zh_tokenizer.decode(t, skip_special_tokens=True)
    try:
      zy = hanzi.to_zhuyin(word)
    except:
      zy = word
      if zy not in error_words:
        error_words.append(zy)
    #print(word,zy)
    try:
      total_zhuyin_tokens[zy] += 1
    except:
      total_zhuyin_tokens[zy] = 1
    sent_zy += zy+'\n'

    #new_tokens = zh_tokenizer(zy)['input_ids']
    #print(new_tokens)
    #new_word = zh_tokenizer.decode(new_tokens, skip_special_tokens=True)
    #print(new_word)
  TEST_sent_to_zhuyin.append(sent_zy[:-1])
  pbar.update(1)
  #if(len(sent_to_zhuyin)>2):
  #  break
#print(total_zhuyin_tokens)
#print(sent_to_zhuyin)

100%|██████████| 36089/36089 [01:24<00:00, 426.23it/s]


In [58]:
ERROR_WORDS = {}
ERROR_WORDS['噢'] = 'ㄡˋ'
ERROR_WORDS['哦'] = 'ㄛˊ'
ERROR_WORDS['喔'] = 'ㄛ'
ERROR_WORDS['喲'] = 'ㄧㄡ'

In [75]:
with tqdm(total = len(sent_to_zhuyin)) as pbar:
 for i in range(len(sent_to_zhuyin)):
  sent = sent_to_zhuyin[i]
  for key in ERROR_WORDS.keys():
    if key in sent:
      while(1):
        try:
          index = sent.index(key)
          sent = sent[:index]+ERROR_WORDS[key]+sent[index+len(key):]
          sent_to_zhuyin[i] = sent
        except:
          break
  pbar.update(1)

100%|██████████| 146661/146661 [00:00<00:00, 372405.68it/s]


In [77]:
with tqdm(total = len(TEST_sent_to_zhuyin)) as pbar:
 for i in range(len(TEST_sent_to_zhuyin)):
  sent = TEST_sent_to_zhuyin[i]
  for key in ERROR_WORDS.keys():
    if key in sent:
      while(1):
        try:
          index = sent.index(key)
          sent = sent[:index]+ERROR_WORDS[key]+sent[index+len(key):]
          TEST_sent_to_zhuyin[i] = sent
        except:
          break
  pbar.update(1)


100%|██████████| 36089/36089 [00:00<00:00, 344222.91it/s]


In [72]:
key = '噢'
sent = 'ㄊㄧㄢㄚ噢ㄋㄧˇㄏㄠˇㄇㄟˇ\nㄋㄧˇㄧㄝˇㄕˋ'
if key in sent:
  index = sent.index(key)
  sent = sent[:index]+ERROR_WORDS[key]+sent[index+len(key):]
  print(sent)

ㄊㄧㄢㄚㄡˋㄋㄧˇㄏㄠˇㄇㄟˇ
ㄋㄧˇㄧㄝˇㄕˋ


In [62]:
print(len(total_zhuyin_tokens))

2751


In [59]:
for zy in ERROR_WORDS.values():
    try:
      total_zhuyin_tokens[zy] += 1
    except:
      total_zhuyin_tokens[zy] = 1

In [61]:
for word in error_words:
  del total_zhuyin_tokens[word]

In [56]:
print(error_words)

['噢', 'o', 'iphone', '##ho', '##go', '##pro', '##ok', 'to', '##ore', 'show', '哦', 'do', '喔', '##orm', 'com', 'micro', 'no', 'non', '##oc', '##ton', '##ora', 'con', '##dot', 'playstation', '##od', 'co', 'pro', '##ion', 'ok', '##zone', 'facebook', '##tion', '##ood', '##no', '##io', '##do', '##o', '##ome', '##ob', '##dia', 'of', 'org', '##tor', 'logo', '##tro', '##zo', 'john', '##os', '##ations', '##osa', 'ol', '##ro', 'android', 'ro', '##so', 'root', 'action', 'nov', '##co', '##ation', '##ction', '##con', '##ol', '##oa', '##ot', 'auto', '##food', '##lvin', 'gov', 'oct', '##op', '##ode', 'go', '##og', 'ago', 'or', '##to', 'on', 'quot', '##tions', 'os', '##on', 'omega', '##uo', 'ho', 'one', '##one', 'not', '##room', '##hoo', 'google', '##ros', 'know', 'come', 'ko', '##oi', '##eo', '##om', 'second', '##lv', '##ko', 'home', '##sol', '##ors', 'op', '##ison', '##tos', 'hero', '喲', 'information', '##of', 'old', 'copyright', '##nos', '##oo', '##row', '##top', 'cho', '##rdon', '##ano', '##vo', 'j

In [32]:
for y in ['ㄅ','ㄆ','ㄇ','ㄈ','ㄉ','ㄊ','ㄋ','ㄌ','ㄍ','ㄎ','ㄏ','ㄐ','ㄑ','ㄒ','ㄓ','ㄔ','ㄕ','ㄖ','ㄗ','ㄘ','ㄙ','ㄧ','ㄨ','ㄩ','ㄚ','ㄛ','ㄜ','ㄝ','ㄞ','ㄟ','ㄠ','ㄡ','ㄢ','ㄣ','ㄤ','ㄥ','ㄦ','ˇ','ˋ','ˊ','˙']:
  yt = zh_tokenizer(y)['input_ids']
  print(y,zh_tokenizer.decode(yt, skip_special_tokens=True))


ㄅ ㄅ
ㄆ ㄆ
ㄇ ㄇ
ㄈ ㄈ
ㄉ ㄉ
ㄊ ㄊ
ㄋ ㄋ
ㄌ ㄌ
ㄍ ㄍ
ㄎ ㄎ
ㄏ ㄏ
ㄐ ㄐ
ㄑ ㄑ
ㄒ ㄒ
ㄓ ㄓ
ㄔ ㄔ
ㄕ ㄕ
ㄖ ㄖ
ㄗ ㄗ
ㄘ ㄘ
ㄙ ㄙ
ㄧ ㄧ
ㄨ ㄨ
ㄩ ㄩ
ㄚ ㄚ
ㄛ ㄛ
ㄜ ㄜ
ㄝ ㄝ
ㄞ ㄞ
ㄟ ㄟ
ㄠ ㄠ
ㄡ ㄡ
ㄢ ㄢ
ㄣ ㄣ
ㄤ ㄤ
ㄥ ㄥ
ㄦ ㄦ
ˇ ˇ
ˋ ˋ
ˊ ˊ
˙ ˙


# Save Files

In [64]:
df = pd.DataFrame({'zhuyin':total_zhuyin_tokens.keys(),'count':total_zhuyin_tokens.values()})
df.to_csv('/content/drive/MyDrive/ZH-KO-TRANSLATOR/CORPUS/zhuyin_tokens_count.csv',index=False)
df.head()

,zhuyin,count
0,ㄒㄧㄝˋ,5191
1,ㄉㄚˋ,27735
2,ㄐㄧㄚ,24204
3,。,166394
4,ㄏㄞˊ,15441


In [65]:
df = pd.DataFrame({'zh':ERROR_WORDS.keys(),'zhuyin':ERROR_WORDS.values()})
df.to_csv('/content/drive/MyDrive/ZH-KO-TRANSLATOR/CORPUS/error_zhuyin_tokens.csv',index=False)
df.head()

,zh,zhuyin
0,噢,ㄡˋ
1,哦,ㄛˊ
2,喔,ㄛ
3,喲,ㄧㄡ


In [79]:
df = pd.DataFrame({'zhuyin':sent_to_zhuyin})
df.to_csv('/content/drive/MyDrive/ZH-KO-TRANSLATOR/CORPUS/ALIGNED/MERGE/TRAIN_ALL_ZHUYIN.csv',index=False)
df.head()

,zhuyin
0,ㄒㄧㄝˋ\nㄒㄧㄝˋ\nㄉㄚˋ\nㄐㄧㄚ\n。
1,ㄏㄞˊ\nㄕˋ\nㄓㄜˋ\nㄕˋ\nㄍㄜˋ\nㄧㄚˋ\nㄓㄡ\nㄋㄢˊ\nㄗ˙\n？
2,（\nㄒㄧㄠˋ\nㄕㄥ\n）\nㄡˋ\n，\nㄗㄞˋ\nㄨㄛˇ\nㄇㄣ˙\nㄐㄧˋ\nㄒㄩˋ...
3,ㄋㄚˋ\nㄐㄧㄤ\nㄏㄨㄟˋ\nㄕˋ\nㄏㄣˇ\nㄘˋ\nㄐㄧ\nㄉㄜ˙\n620\nㄧㄥ\...
4,ㄦˊ\nㄨㄛˇ\nㄋㄜ˙\n，\nㄗㄞˋ\nㄨㄛˇ\nㄏㄣˇ\nㄓㄤˇ\nㄉㄜ˙\nㄑㄧㄥ\...


In [80]:
df = pd.DataFrame({'zhuyin':TEST_sent_to_zhuyin})
df.to_csv('/content/drive/MyDrive/ZH-KO-TRANSLATOR/CORPUS/ALIGNED/MERGE/TEST_ALL_ZHUYIN.csv',index=False)
df.head()

,zhuyin
0,ㄨㄛˇ\nㄒㄧㄤ\nㄒㄧㄣˋ\nㄖㄨˊ\nㄍㄨㄛˇ\nㄋㄧˇ\nㄍㄟˇ\nㄊㄚ\nㄇㄣ˙\n...
1,」\nㄒㄧㄢˋ\nㄗㄞˋ\nㄏㄨㄟˊ\nㄉㄠˋ\nㄨㄛˇ\nㄗㄞˋ\nㄗㄨㄟˋ\nㄔㄨ\nㄙ...
2,ㄧㄣ\nㄘˇ\nㄓㄜˋ\nㄒㄧㄝ\nㄇㄨˋ\nㄗㄠˋ\nㄐㄧㄢˋ\nㄩㄢˊ\nㄅㄨˋ\nㄌㄨ...
3,ㄉㄢˋ\nㄐㄧㄡˋ\nㄙㄨㄢˋ\nㄓˇ\nㄧㄡˇ\nㄋㄚˋ\nㄒㄧㄝ\n30\nㄘㄥˊ\nㄧ...
4,ㄗㄨㄥˇ\nㄐㄧㄝˊ\nㄧˇ\nㄕㄤˋ\n，\nㄨㄢˊ\nㄑㄩㄢˊ\nㄇㄨˋ\nㄗㄠˋ\nㄉ...
